In [2]:
train = pd.read_csv('../input/playground-series-s4e7/train.csv')
test = pd.read_csv('../input/playground-series-s4e7/test.csv')
ss = pd.read_csv('../input/playground-series-s4e7/sample_submission-2.csv')

In [4]:
import joblib
dl135 = joblib.load('nn_oofs/denselight_5fold_oof_test_089135.jbl')
dl161 = joblib.load('nn_oofs/denselight_5fold_oof_test_089161.jbl')

ft150 = joblib.load('nn_oofs/fttransformer_5fold_oof_test_089150.jbl')
rn157 = joblib.load('nn_oofs/resnet_5fold_oof_test_089157.jbl')

In [24]:
HOLDOUT = False

In [25]:
if HOLDOUT:
    X_train, X_val = train_test_split(train, test_size=0.2, random_state=42, shuffle=True, stratify=train.Response)
else:
    X_train, X_val = train, test

In [35]:
X_train.loc[:, 'ft150'] = ft150[0]
X_train.loc[:, 'rn157'] = rn157[0]
X_train.loc[:, 'dl161'] = dl161[0]
X_train.loc[:, 'dl135'] = dl135[0]

X_val.loc[:, 'ft150'] = ft150[1]
X_val.loc[:, 'rn157'] = rn157[1]
X_val.loc[:, 'dl161'] = dl161[1]
X_val.loc[:, 'dl135'] = dl135[1]

In [36]:
X_train

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,ft150,rn157,dl161,dl135
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0,0.260343,0.330062,0.310721,0.328448
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1,0.466775,0.495850,0.502867,0.501508
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0,0.000228,0.000112,0.000068,0.000182
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0,0.141679,0.175906,0.159691,0.167480
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0,0.000313,0.000266,0.000029,0.000041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,11504793,Male,48,1,6.0,0,1-2 Year,Yes,27412.0,26.0,218,0,0.474919,0.396422,0.464265,0.410795
11504794,11504794,Female,26,1,36.0,0,< 1 Year,Yes,29509.0,152.0,115,1,0.214196,0.301742,0.282953,0.337362
11504795,11504795,Female,29,1,32.0,1,< 1 Year,No,2630.0,152.0,189,0,0.000433,0.000215,0.000233,0.000342
11504796,11504796,Female,51,1,28.0,0,1-2 Year,Yes,48443.0,26.0,274,1,0.192171,0.182069,0.186144,0.209001


In [37]:
task = Task('binary') #‘binary’ 
automl = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions1 = automl.fit_predict(
    X_train,
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

val_predict = automl.predict(X_val)
if HOLDOUT:
    roc_auc_score(X_val.Response, val_predict.data)

[23:46:39] Stdout logging level is DEBUG.
[23:46:39] Task: binary

[23:46:39] Start automl preset with listed constraints:
[23:46:39] - time: 2160000.00 seconds
[23:46:39] - CPU: 12 cores
[23:46:39] - memory: 16 GB

[23:46:39] Train data shape: (11504798, 16)

[23:46:53] Feats was rejected during automatic roles guess: []
[23:46:54] Layer 1 train process start. Time left 2159984.49 secs
[23:53:39] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[23:53:39] Training params: {'task': 'train', 'learning_rate': 0.05, 'num_leaves': 244, 'feature_fraction': 0.7, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'max_depth': -1, 'verbosity': -1, 'reg_alpha': 1, 'reg_lambda': 0.0, 'min_split_gain': 0.0, 'zero_as_missing': False, 'num_threads': 12, 'max_bin': 255, 'min_data_in_bin': 3, 'num_trees': 2000, 'early_stopping_rounds': 100, 'random_state': 42}
[23:53:40] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[23:53:43] Training until validation scores don't improve for 100 round

In [38]:
roc_auc_score(X_train.Response, out_of_fold_predictions1.data[:, 0])

0.892746022465867

## 2 binary model

In [39]:

task = Task('binary') #‘binary’ 
automl1 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions1_1 = automl1.fit_predict(
    X_train[X_train.Previously_Insured==0], 
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)


automl2 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)
out_of_fold_predictions2_1 = automl2.fit_predict(
    X_train[X_train.Previously_Insured==1],
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

val_predict1 = automl1.predict(X_val[X_val.Previously_Insured==0])
val_predict2 = automl2.predict(X_val[X_val.Previously_Insured==1])


y_pred = pd.Series(index=X_val.index)
y_pred.iloc[X_val.Previously_Insured==0] = val_predict1.data[:, 0]
y_pred.iloc[X_val.Previously_Insured==1] = val_predict2.data[:, 0]

oof1 = pd.Series(index=X_train.index)
oof1.iloc[X_train.Previously_Insured==0] = out_of_fold_predictions1_1.data[:, 0]
oof1.iloc[X_train.Previously_Insured==1] = out_of_fold_predictions2_1.data[:, 0]

if HOLDOUT:
    roc_auc_score(X_val.Response, y_pred)

[00:01:39] Stdout logging level is DEBUG.
[00:01:39] Task: binary

[00:01:39] Start automl preset with listed constraints:
[00:01:39] - time: 2160000.00 seconds
[00:01:39] - CPU: 12 cores
[00:01:39] - memory: 16 GB

[00:01:39] Train data shape: (6178116, 16)

[00:01:49] Feats was rejected during automatic roles guess: []
[00:01:49] Layer 1 train process start. Time left 2159989.54 secs
[00:05:22] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[00:05:22] Training params: {'task': 'train', 'learning_rate': 0.05, 'num_leaves': 244, 'feature_fraction': 0.7, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'max_depth': -1, 'verbosity': -1, 'reg_alpha': 1, 'reg_lambda': 0.0, 'min_split_gain': 0.0, 'zero_as_missing': False, 'num_threads': 12, 'max_bin': 255, 'min_data_in_bin': 3, 'num_trees': 2000, 'early_stopping_rounds': 100, 'random_state': 42}
[00:05:22] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[00:05:24] Training until validation scores don't improve for 100 rounds

In [40]:
roc_auc_score(X_train.Response, oof1)

0.8914516316380822

In [41]:

task = Task('binary') #‘binary’ 
automl1_1 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions1_2 = automl1_1.fit_predict(
    X_train[X_train.Vehicle_Damage=='Yes'], 
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)


automl2_1 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)
out_of_fold_predictions2_2 = automl2_1.fit_predict(
    X_train[X_train.Vehicle_Damage=='No'],
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

val_predict1_2 = automl1_1.predict(X_val[X_val.Vehicle_Damage=='Yes'])
val_predict2_2 = automl2_1.predict(X_val[X_val.Vehicle_Damage=='No'])


y_pred2 = pd.Series(index=X_val.index)
y_pred2.iloc[X_val.Vehicle_Damage=='Yes'] = val_predict1_2.data[:, 0]
y_pred2.iloc[X_val.Vehicle_Damage=='No'] = val_predict2_2.data[:, 0]

oof2 = pd.Series(index=X_train.index)
oof2.iloc[X_train.Vehicle_Damage=='Yes'] = out_of_fold_predictions1_2.data[:, 0]
oof2.iloc[X_train.Vehicle_Damage=='No'] = out_of_fold_predictions2_2.data[:, 0]

if HOLDOUT:
    roc_auc_score(X_val.Response, y_pred2)

[00:19:48] Stdout logging level is DEBUG.
[00:19:48] Task: binary

[00:19:48] Start automl preset with listed constraints:
[00:19:48] - time: 2160000.00 seconds
[00:19:48] - CPU: 12 cores
[00:19:48] - memory: 16 GB

[00:19:48] Train data shape: (5783229, 16)

[00:19:57] Feats was rejected during automatic roles guess: []
[00:19:57] Layer 1 train process start. Time left 2159990.43 secs
[00:23:15] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[00:23:15] Training params: {'task': 'train', 'learning_rate': 0.05, 'num_leaves': 244, 'feature_fraction': 0.7, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'max_depth': -1, 'verbosity': -1, 'reg_alpha': 1, 'reg_lambda': 0.0, 'min_split_gain': 0.0, 'zero_as_missing': False, 'num_threads': 12, 'max_bin': 255, 'min_data_in_bin': 3, 'num_trees': 2000, 'early_stopping_rounds': 100, 'random_state': 42}
[00:23:15] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[00:23:17] Training until validation scores don't improve for 100 rounds

In [42]:
roc_auc_score(X_train.Response, oof2)

0.8927603717013254

In [43]:
if HOLDOUT:
    roc_auc_score(X_val.Response, y_pred+y_pred2)

In [44]:
roc_auc_score(X_train.Response, oof1+oof2)

0.892033343062512

In [45]:
Xt1 = X_train[X_train.Previously_Insured==0]
Xt1 = Xt1[Xt1.Vehicle_Damage=='Yes']

Xt2 = X_train[X_train.Previously_Insured==0]
Xt2 = Xt2[Xt2.Vehicle_Damage=='No']

Xt3 = X_train[X_train.Previously_Insured==1]
Xt3 = Xt3[Xt3.Vehicle_Damage=='Yes']

Xt4 = X_train[X_train.Previously_Insured==1]
Xt4 = Xt4[Xt4.Vehicle_Damage=='No']

Xv1 = X_val[X_val.Previously_Insured==0]
Xv1 = Xv1[Xv1.Vehicle_Damage=='Yes']

Xv2 = X_val[X_val.Previously_Insured==0]
Xv2 = Xv2[Xv2.Vehicle_Damage=='No']

Xv3 = X_val[X_val.Previously_Insured==1]
Xv3 = Xv3[Xv3.Vehicle_Damage=='Yes']

Xv4 = X_val[X_val.Previously_Insured==1]
Xv4 = Xv4[Xv4.Vehicle_Damage=='No']

task = Task('binary') #‘binary’ 

automl1_3 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions1_3 = automl1_3.fit_predict(
    Xt1, 
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

automl2_3 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions2_3 = automl2_3.fit_predict(
    Xt2, 
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

automl3_3 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions3_3 = automl3_3.fit_predict(
    Xt3,
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

automl4_3 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions4_3 = automl4_3.fit_predict(
    Xt4,
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

val_predict1_3 = automl1_3.predict(Xv1)
val_predict2_3 = automl2_3.predict(Xv2)
val_predict3_3 = automl3_3.predict(Xv3)
val_predict4_3 = automl4_3.predict(Xv4)

y_pred3 = pd.Series(index=X_val.index)
y_pred3.loc[Xv1.index] = val_predict1_3.data[:, 0]
y_pred3.loc[Xv2.index] = val_predict2_3.data[:, 0]
y_pred3.loc[Xv3.index] = val_predict3_3.data[:, 0]
y_pred3.loc[Xv4.index] = val_predict4_3.data[:, 0]

oof3 = pd.Series(index=X_train.index)
oof3.loc[Xt1.index] = out_of_fold_predictions1_3.data[:, 0]
oof3.loc[Xt2.index] = out_of_fold_predictions2_3.data[:, 0]
oof3.loc[Xt3.index] = out_of_fold_predictions3_3.data[:, 0]
oof3.loc[Xt4.index] = out_of_fold_predictions4_3.data[:, 0]


if HOLDOUT:
    roc_auc_score(X_val.Response, y_pred3.values)

[00:33:57] Stdout logging level is DEBUG.
[00:33:57] Task: binary

[00:33:57] Start automl preset with listed constraints:
[00:33:57] - time: 2160000.00 seconds
[00:33:57] - CPU: 12 cores
[00:33:57] - memory: 16 GB

[00:33:57] Train data shape: (5504102, 16)

[00:34:06] Feats was rejected during automatic roles guess: []
[00:34:07] Layer 1 train process start. Time left 2159990.26 secs
[00:37:23] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[00:37:23] Training params: {'task': 'train', 'learning_rate': 0.05, 'num_leaves': 244, 'feature_fraction': 0.7, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'max_depth': -1, 'verbosity': -1, 'reg_alpha': 1, 'reg_lambda': 0.0, 'min_split_gain': 0.0, 'zero_as_missing': False, 'num_threads': 12, 'max_bin': 255, 'min_data_in_bin': 3, 'num_trees': 2000, 'early_stopping_rounds': 100, 'random_state': 42}
[00:37:23] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[00:37:25] Training until validation scores don't improve for 100 rounds

In [46]:
roc_auc_score(X_train.Response, oof3)

0.8922743333965447

In [47]:
if HOLDOUT:
    roc_auc_score(X_val.Response, y_pred.values + y_pred2.values + y_pred3.values)

In [50]:
joblib.dump(
    {'m1_full_train': automl,
    'm2': {'Previously_Insured0': automl1, 'Previously_Insured1': automl2},
    'm3': {'Vehicle_DamageY': automl1_1, 'Vehicle_DamageN':automl2_1},
    'm4': {'PI0VDY': automl1_3, 'PI0VDN':automl2_3, 'PI1VDY':automl3_3, 'PI1VDN':automl4_3}},
     'gbdts_stack_with_nn.jbl')

['gbdts_stack_with_nn.jbl']